# Single-Qubit Calibration


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial introduces how to calibrate the qubit transition frequency, $\pi$ pulse, relaxation times $T_1$ and $T_2$ of a single qubit using Quanlse. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Single-qubit simulator initialization
- Qubit frequency calibration
- $\pi$ pulse calibration by Rabi oscillation
- Calibrate $T_1$ by longitudinal relaxation
- Calibrate $T_2$ by Ramsey oscillation
- Summary

## Introduction

Due to the imperfection in the manufacturing process and the needs of practical applications, each superconducting qubit has unique properties such as the qubit frequency, the qubit relaxation times. Therefore, each qubit requires a thorough calibration. By calibration, we perform a series of operations on the qubit and measure its response. Information regarding the qubit's properties can then be obtained, including the qubit frequency, the corresponding $\pi$ pulse parameters, the qubit relaxation times $T_1$ and $T_2$. The qubit frequency is the driving frequency of pulse signal to realize single qubit gate. The relaxation times are the duration for which a qubit keeps its information. The longer the relaxation time, the better the quality of the qubit and the longer the time available for operation.

## Preparation

We first import the necessary packages from Quanlse and other commonly used Python libraries.

In [ ]:
from Quanlse.Simulator.PulseSim1Q import pulseSim1Q
from Quanlse.Calibration.SingleQubit import qubitSpec, ampRabi, fitRabi, longRelax, ramsey, fitRamsey

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 5)

from numpy import array, pi, exp
from scipy.signal import find_peaks

## Single-qubit simulator initialization

Then we proceed to define the system. Since we are dealing with a one-qubit system, we can directly import the built-in simulator `pulseSim1Q()` in Quanlse. The function `pulseSim1Q()` takes two parameters: `dt` is the AWG sampling time, and `frameMode` indicates which frame we would like to perform our experiment in. With the simulator object created, we can specify a few parameters such as the qubit frequency and qubit relaxation times $T_1$ and $T_2$. While here we do have prior knowledge of the qubit's properties, we will treat the system as a black box in the following sessions.

In [ ]:
# AWG sampling time
dt = 0.01

# Instantiate the simulator object
model = pulseSim1Q(dt=dt, frameMode='lab')

# Define system parameters
model.qubitFreq = {0: 5.212 * (2 * pi)}
model.T1 = {0 : 2000}
model.T2 = {0 : 600}

## Qubit frequency calibration

Before we calibrate any other qubit parameters, we need to identify the frequency of the qubit we are working with. Once we obtain the qubit frequency, we can correctly set the frequency of the local oscillator so that the applied pulse is in resonance with the qubit.

To find the qubit frequency, we apply a pulse of reasonable amplitude to the qubit while varying the frequency of the local oscillator. When the frequency is in resonance with the qubit, the pulse can excite the qubit from its ground state. In actual experiments, the chip is handed to the experimenters with an approximation of the qubit frequency. Thus, we will first scan a range of frequencies centered at the given frequency, and pin down the accurate qubit frequency.

We first scan a broader range from 4.6 GHz to 5.8 GHz. Here, we can use the function `qubitSpec()` from the calibration module. This function takes a pulseModel `model` , a range of frequency `freqRange` , sample size `sample` , pulse amplitude `amp` and duration `t`. The function then returns the scanned frequency list and the corresponding excited state populations.

In [ ]:
# Define frequency range
freqRange = [4.1 * (2 * pi), 5.9 * (2 * pi)]

# Scan qubit frequency spectrum
freqList, popList = qubitSpec(pulseModel=model, freqRange=freqRange, sample=50, amp=0.9, t=20)

Now we plot the excited state populations with respect to the LO frequencies.

In [ ]:
# Convert unit
freq = [x / (2 * pi) for x in freqList]

# Plot population graph
plt.plot(freq, popList)
plt.title("Frequency spectrum", size=17)
plt.xlabel("LO frequency (GHz)", size=15)
plt.ylabel(r"$|1\rangle$ population)", size=15)
plt.show()

From the graph we can see that the qubit frequency is somewhere between 5.1GHz and 5.3GHz. To acquire a more accurate value, we run another scan in a narrower range.

In [ ]:
# Define new frequency range
nFreqRange = [5.1 * (2 * pi), 5.3 * (2 * pi)]

# Scan qubit frequency spectrum
nFreqList, nPopList = qubitSpec(model, nFreqRange, 30, 0.9, 20)

In [ ]:
# Convert unit
nFreq = [x / (2 * pi) for x in nFreqList]

# Plot population graph
plt.plot(nFreq, nPopList)
plt.title("Frequency spectrum", size=17)
plt.xlabel("LO frequency (GHz)", size=15)
plt.ylabel(r"$|1\rangle$ population)", size=15)
plt.show()

Then, we use the function `find_peak()` from `scipy` to locate the frequency corresponding to the maximum value on the peak.

In [ ]:
# Find peak
peak = find_peaks(nPopList, height=0.3)[0][0]
qubitFreq = nFreq[peak]

# Plot peak
plt.plot(nFreq, nPopList)
plt.title(f'Qubit frequency: {round(qubitFreq, 6)} GHz', size=17)
plt.plot(nFreq[peak], nPopList[peak], 'x', mfc=None, mec='red', mew=2, ms=8)
plt.xlabel('Frequency (GHz)', size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.show()

We find the qubit frequency to be 5.217241 GHz shown in the graph above.

## $\pi$ pulse calibration by Rabi oscillation

With the qubit frequency identified, we can now calibrate the $\pi$ and $\pi/2$ pulse of the system. To do so, we perform a Rabi oscillation. There are generally two ways of performing Rabi oscillation: we can scan either the pulse amplitudes or durations while having the other fixed. With an appropriate range chosen, the population of the excited (or ground) state will oscillate as a sinuous function. To run a rabi oscillation in Quanlse, we import the function `ampRabi()` from the `calibration` module. The function `ampRabi()` takes 4 arguments: pulseModel `model` , amplitudes' range `ampRange` , pulse duration `tg` and the sample size `sample`. The function then returns a list of amplitudes scanned and a list of corresponding excited state populations.

Note that the `calibration` module also includes function `tRabi()` which scans the $\pi$ pulse duration. This function is quite similar to `ampRabi()`, thus will not be demonstrated here.

In [ ]:
# Define amplitude range
ampRange = [0, 6]

# Scan different amplitudes for Rabi oscillation
ampList, popList = ampRabi(pulseModel=model, pulseFreq=qubitFreq * 2 * pi,
                           ampRange=ampRange, tg=20, sample=50)

We can plot the excited state populations with respect to the pulse amplitudes.

In [ ]:
# Plot Rabi Oscillation with different amplitudes
plt.plot(ampList, popList, '.')
plt.title("Rabi Oscillation", size=17)
plt.xlabel('Amplitude', size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.show()

Then, using the `fitRabi()` function from the `calibration` module, we can fit the curve with a cosine function and obtain the amplitudes for the $\pi$ and $\pi/2$ pulse. We input the list of amplitudes `ampList` as X axis and the list of populations `popList` as Y axis. The fitting function takes the form: $y = a \cdot cos(b \cdot x+c)+d$. The function `fitRabi()` returns two values, first of which is the $\pi/2$ pulse amplitude and the second being that of a $\pi$ pulse.

In [ ]:
# Fit Rabi
halfPiAmp, piAmp = fitRabi(popList=popList, xList=ampList)
print("Pi/2 pulse amplitude: ", halfPiAmp)
print("Pi pulse amplitude: ", piAmp)

## Calibrate $T_1$ by longitudinal relaxation

Now we have obtained the $\pi$ and $\pi/2$ pulse configurations. We can further calibrate the qubit relaxation times $T_1$ and $T_2$. To obtain $T_1$, we first apply a $\pi$-pulse on the qubit and find the time when the population of the excited state decays to $1/e$ \[1\].

To excite the qubit to the excited state and to observe the longitudinal relaxation, we can use the function `longRelax()` from the `Quanlse.Calibration.SingleQubit` module. The function arguments include: a pulseModel object `model`, AWG sampling time `dt`, qubit frequency, $\pi$ Pulse amplitude `piAmp` and duration(`20`), maximum idling time `maxIdle` and an initial value for the fitting function `initFit`. The function would then run the simulation and fit the population curve with function $y = e^{-x/T_1}$ .

In [ ]:
# Longitudinal relaxation on a qubit
T1, tList, experimental, fitted = longRelax(pulseModel=model, dt=dt, pulseFreq=qubitFreq * 2 * pi,
                                            piAmp=piAmp, piLen=20, maxIdle=4000, initFit=[1500])

The function returns the estimated $T_1$, a list of idling times, a list of experimentally obtained populations and a list of populations estimated by the fitting function. We can plot the resulting populations with respect to the idling time:

In [ ]:
# Print estimated T1
print("Estimated T1: ", T1, "ns")

# Plot fit result
plt.plot(tList, experimental, "+", label="Experiment")
plt.plot(tList, fitted, "r", label="Fitted", linewidth=2.)
plt.legend()
plt.xlabel("Idling time", size=15)
plt.ylabel(r'$|1\rangle$ population', size=15)
plt.title("Longitudinal Relaxation", size=17)
plt.show()

## Calibrate $T_2$ by Ramsey oscillation

To find $T_2$ which characterizes the qubit's depolarization, we perform a Ramsey oscillation on the qubit. To do so, we first apply a $\pi/2$ pulse at drive frequency slightly detuned from the qubit frequency. After an idling time of $t_{\rm idle}$ , we apply another $\pi/2$ pulse and measure the excited state population of the qubit \[2\]. Now the measurement outcome depends on the phase of the quantum state accumulated in the idling time $t_{\rm idle}$.

To perform a Ramsey experiment on the qubit, we use the function `ramsey()` in the `Quanlse.Calibration.SingleQubit` module. This function takes a pulseModel object `model` , qubitFreq `qubitFreq`, $\pi/2$ pulse duration `tg`, $\pi/2$ pulse amplitude `halfPiAmp`, sample number `sample`, maximum idling time `maxTime` and the amplitude of the detuning pulse `detuning` (This simulation may take some time and you can reduce it by setting lesser sample points with worse simulation result).

In [ ]:
# Scan different idle time for Ramsey oscillation
tList, popList = ramsey(pulseModel=model, pulseFreq=5.21 * 2 * pi, tg=20, x90=1.013,
                        sample=50, maxTime=600, detuning=0.07)

The function then returns a list of idling times and a corresponding list of populations. We can fit the results using the function `fitRamsey()` which takes the system's $T_1$ `t1`, a list of populations `popList`, a list of idling times `tList` and an amplitude of the detuning wave `detuning`. This function then fits the curve with function $y = \frac{1}{2} cos(a\cdot x)e^{-b\cdot x} + 0.5$. With the coefficients found, we can find $T_2$ using the equation: $T_2 = \frac{1}{(b-\frac{1}{2a})}$ .

In [ ]:
# Fit Ramsey
T2, fitted = fitRamsey(2000, popList=popList, tList=tList, detuning=0.07)

`fitRamsey()` returns the estimated $T_2$ value and a list of the populations on the fitted curve. We can plot the excited state population with respect to the idling time as shown below:

In [ ]:
# Print estimated T2
print("Estimated T2: ", T2, " ns")

# Plot fit result
plt.plot(tList, popList, '.')
plt.plot(tList, fitted)
plt.plot(tList, list(exp(- (1 / 600 + 1 / (2 * 2000)) * array(tList)) * 0.5 + 0.5))
plt.xlabel("Idling time (ns)", size=15)
plt.ylabel(r"$|1\rangle$ population", size=15)
plt.title("Ramsey Experiment", size=17)
plt.show()

## Summary

This tutorial introduces how to use Quanlse for calibrations of single qubit frequency、$\pi$ pulse、relaxation time $T_1$ and dephasing time $T_2$. The users could follow this link [tutorial-single-qubit-calibration.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-single-qubit-calibration.ipynb) to the GitHub page of this Jupyter Notebook document and run this program for themselves. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result. At the same time, after reading this tutorial, readers should have a general understanding of the single qubit calibration process. 


## Reference

\[1\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://doi.org/10.1063/1.5089550)

\[2\] [Ramsey, Norman F. "A molecular beam resonance method with separated oscillating fields." *Physical Review* 78.6 (1950): 695.](https://doi.org/10.1103/PhysRev.78.695)